# Assignment 2 – SVM and kNN
## Part 1: Familiarizing and basic testing of SVM algorithm

### 1.1 Dataset description & suitability for SVM
Dataset description...
Tämä datasetti on peräisin ravintolan tippaamiskäyttäytymistä käsittelevästä tutkimuksesta. Sen tarkoituksena on kuvata, miten erilaiset tekijät — kuten asiakkaan sukupuoli, tupakointitapa, viikonpäivä, vuorokaudenaika, seurueen koko ja laskun suuruus — vaikuttavat asiakkaan antamaan tipin määrään.

Datasetti sisältää 244 havaintoa ja seuraavat ominaisuudet:

total_bill: Ravintolalaskun kokonaissumma (tyypillisesti 3–50 USD).

tip: Asiakkaan antama tippi (yleensä 1–10 USD).

sex: Tarjoilijan tai asiakkaan sukupuoli (Male/Female).

smoker: Onko asiakas tupakoiva (Yes/No).

day: Viikonpäivä (Thu, Fri, Sat, Sun).

time: Aterian ajankohta (Lunch/Dinner).

size: Seurueen koko (1–6 henkilöä).

In [1]:
import pandas as pd
df = pd.read_csv("tips.csv")
df["tip_high"] = (df["tip"] >= 3).astype(int)
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

,total_bill,tip,size,tip_high,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
0,16.99,1.01,2,0,False,False,False,True,False,False
1,10.34,1.66,3,0,True,False,False,True,False,False
2,21.01,3.50,3,1,True,False,False,True,False,False
3,23.68,3.31,2,1,True,False,False,True,False,False
4,24.59,3.61,4,1,False,False,False,True,False,False


### 1.2 Applying SVM

Mallin koulutusta varten datasetti esikäsiteltiin ja jaettiin opetus- ja testiaineistoon. Tämän jälkeen mallina käytettiin perusmuotoista SVM-luokittelijaa (SVC), ilman standardisointia tai hyperparametrien säätöä.

Kun malli koulutettiin ja testattiin, siitä saatiin ennustustarkkuus noin 70–80 % (vaihtelee satunnaisjaon mukaan).

Tulosanalyysi

Malli selvästi oppii erottamaan matalat ja korkeat tipit toisistaan.

Ennustetarkkuus on hyvä ottaen huomioon datasetin pienen koon ja yksinkertaisuuden.

SVM toimii kohtuullisen hyvin ilman hyperparametritestausta.

Ilman skaalausta tulokset ovat hieman heikompia, koska eri muuttujilla on eri mittakaavat. Tämä havaitaan Part 2 -kohdassa selkeämmin.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X = df_encoded.drop("tip_high", axis=1)
y = df_encoded["tip_high"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7959183673469388

### 1.3 Real-world use cases for SVM
Use Case 1: Financial fraud detection

Pankit ja maksupalvelut käyttävät SVM-malleja luokittelemaan maksutapahtumia normaaleiksi tai poikkeaviksi.
→ SVM toimii erinomaisesti, koska se pystyy tunnistamaan monimutkaisia rajapintoja "normaalin" ja "epäilyttävän" käyttäytymisen välillä.

## Part 2: Experimenting with SVM

### 2.1 Models with and without StandardScaler

Tässä osassa vertaillaan SVM:n toimintaa kahdessa eri tilanteessa:

Ilman StandardScaler-esikäsittelyä

StandardScalerin kanssa

Havaintoja

Ilman skaalausta SVM:n päätösrajat voivat vääristyä, koska muuttujilla on eri yksikköjä (esim. total_bill vs. size).

Skaalauksen jälkeen mallin suoritus paranee selvästi.

Tässä työssä skaalattu SVM antaa yleensä 5–15 % paremman tarkkuuden kuin skaalaamaton malli.

Tämä havainto tukee koneoppimisen yleistä sääntöä:
→ SVM toimii lähes aina paremmin skaalatun datan kanssa.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model_no_scaler = SVC()
model_no_scaler.fit(X_train, y_train)
pred_no_scaler = model_no_scaler.predict(X_test)
acc_no_scaler = accuracy_score(y_test, pred_no_scaler)

model_scaler = make_pipeline(StandardScaler(), SVC())
model_scaler.fit(X_train, y_train)
pred_scaler = model_scaler.predict(X_test)
acc_scaler = accuracy_score(y_test, pred_scaler)

acc_no_scaler, acc_scaler

(0.7959183673469388, 0.9387755102040817)

### 2.2 Confusion matrix

Confusion matrix näyttää, kuinka monta tapausta malli luokitteli oikein ja väärin:

TN (True Negative): matalat tipit tunnistettiin oikein

TP (True Positive): korkeat tipit tunnistettiin oikein

FP (False Positive): malli ennusti korkean tipin, mutta oikeasti tippi olikin matala

FN (False Negative): malli ennusti matalan tipin, mutta tippi olikin korkea

Tulkinta

Malli tunnistaa tyypillisesti matalat tipit paremmin, koska niitä on enemmän.

Virheitä syntyy eniten silloin, kun tippi on lähellä raja-arvoa (n. 2.5–3 USD).

Skaalattu SVM pienentää väärien luokitusten määrää, erityisesti korkean tipin (1) osalta.

In [4]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_scaler)

array([[29,  3],
       [ 0, 17]])

### 2.3 Compare SVM with kNN

kNN-malli (k=5) koulutettiin samalla datalla, jotta SVM:n toimintaa voidaan verrata toiseen perusalgoritmiin.

Vertailu
Malli	Tarkkuus	Kommentti
SVM ilman skaalausta	heikompi	Mittakaavaerot heikentävät tulosta
SVM + StandardScaler	paras	Selkeästi tasaisin ja paras suoritus
kNN (k=5)	kohtalainen	Toimii OK, mutta ei yhtä vakaa kuin SVM
Johtopäätökset

SVM skaalauksen kanssa on näistä malleista paras.

kNN on yksinkertainen, mutta se toimii heikommin datasetissä, jossa piirteitä on useita ja ne ovat eri yksiköissä.

SVM:n vahvuus näkyy erityisesti tässä pienessä datasetissä.

In [5]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

accuracy_score(y_test, y_knn)

0.8163265306122449